### Object Tracking in Video Feeds for Traffic Monitoring

Built an object tracking system to monitor and count vehicles and other objects in a video feed

### Libraries

In [3]:
#!pip install opencv-python numpy torch torchvision yolov5 filterpy

In [11]:
import cv2
import requests
import numpy as np
import pandas
import torchvision
from filterpy.kalman import KalmanFilter
from sort import Sort
import torch

##### Loading the Pre-trained Object Detection Model YOLOv3

In [2]:
#git clone https://github.com/ultralytics/yolov5.git
#cd yolov5
#pip install -r requirements.txt

In [12]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load small YOLOv5 model

# Define a function for object detection
def detect_objects(frame):
    results = model(frame)
    detections = results.pandas().xyxy[0]  # Convert to pandas dataframe
    return detections

Using cache found in C:\Users\STUTERN/.cache\torch\hub\ultralytics_yolov5_master


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'setuptools>=70.0.0'] not found, attempting AutoUpdate...
requirements:  AutoUpdate skipped (offline)


YOLOv5  2024-11-23 Python-3.10.11 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


#### Loading and processing Video Feed

In [13]:
video_path = "traffic_video.mp4"
cap = cv2.VideoCapture(video_path)

##### Iterate through video frames

In [14]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for faster processing
    resized_frame = cv2.resize(frame, (640, 360))

    # Detect objects
    detections = detect_objects(resized_frame)

    # Draw bounding boxes
    for _, row in detections.iterrows():
        x1, y1, x2, y2, conf, cls = map(int, row[['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']])
        label = f"{row['name']} {conf:.2f}"
        cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(resized_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Traffic Feed", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


##### Implementing Tracking Object

In [ ]:
#git clone https://github.com/abewley/sort.git
#cd sort
#pip install .

##### Integrating SORT for tracking

In [15]:
# Initialize SORT tracker
tracker = Sort()

def track_objects(detections):
    tracked_objects = tracker.update(detections[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].values)
    return tracked_objects

##### Combine detection and tracking

In [23]:
# Initialize SORT tracker
tracker = Sort()

# Counting line and vehicle count
counting_line = 300  # Y-coordinate for counting line
vehicle_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (640, 360))
    detections = detect_objects(resized_frame)

    # Convert to NumPy for SORT
    detection_array = detections[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].values
    tracked_objects = track_objects(detection_array)

    # Draw tracked objects
    for obj in tracked_objects:
        x1, y1, x2, y2, obj_id = map(int, obj)
        label = f"ID {obj_id}"
        cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(resized_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Counting vehicles that cross the line
        center_y = (y1 + y2) // 2
        if center_y > counting_line - 10 and center_y < counting_line + 10:
            vehicle_count += 1

    # Display vehicle count on the frame
    cv2.putText(resized_frame, f"Count: {vehicle_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Show the frame with tracked objects
    cv2.imshow("Tracked Traffic Feed", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
